<a href="https://colab.research.google.com/github/Athu7/Fake-news-classification-/blob/main/Fake_news_classifier_(deep_learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Kaggle fake news classifier /fake news data.csv")

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df = df.dropna()

In [ ]:
## Get the independent features 

X = df.drop('label', axis = 1)

In [ ]:
X.shape

(18285, 4)

In [ ]:
y = df['label']

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf 

In [ ]:
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
# Vocabulary size 
voc_size = 5000

#**One hot representation**

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace =True)

In [ ]:
##Data preprocessing 
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus =[]
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[1485, 755, 4960, 1152, 4015, 208, 4648, 2922, 720, 1002],
 [595, 3545, 4304, 2463, 878, 2726, 54],
 [2583, 2373, 4843, 6],
 [3373, 160, 2520, 4470, 2995, 2663],
 [1778, 878, 2767, 466, 1620, 2426, 878, 4717, 2568, 4828],
 [629,
  961,
  1307,
  476,
  450,
  3910,
  3876,
  1562,
  4865,
  31,
  3146,
  4339,
  2181,
  2648,
  54],
 [48, 4806, 4375, 4119, 4419, 2747, 643, 4209, 4555, 3494, 2690],
 [4804, 1826, 4373, 1817, 4351, 4472, 3910, 4898, 4555, 3494, 2690],
 [4354, 4287, 3100, 1818, 1886, 4389, 4293, 2034, 3910, 471],
 [4784, 3036, 4691, 4763, 2801, 2746, 3419, 2446],
 [3128, 3171, 2005, 3922, 560, 1331, 3220, 4904, 396, 1000, 1995],
 [4470, 1737, 4015, 4389, 3910, 4351],
 [240, 2847, 2483, 857, 348, 2825, 37, 3904, 4622],
 [822, 4488, 1412, 3869, 1616, 4208, 2517, 4555, 3494, 2690],
 [4987, 2523, 1754, 537, 614, 4555, 3494, 2690],
 [524, 2756, 149, 3515, 1625, 650, 1769, 400, 4295, 1362],
 [1614, 3115, 3545],
 [2547, 3906, 1746, 3276, 3910, 2213, 38, 54],
 [1656, 2056, 4304, 

#**Embedding representation**

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 2922  720 1002]
 [   0    0    0 ...  878 2726   54]
 [   0    0    0 ... 2373 4843    6]
 ...
 [   0    0    0 ... 4555 3494 2690]
 [   0    0    0 ...  784 4233 3371]
 [   0    0    0 ... 3154  142 1025]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1485,
        755, 4960, 1152, 4015,  208, 4648, 2922,  720, 1002], dtype=int32)

In [ ]:
#Creating the model

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add((1, activation = 'sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.33, random_state = 42)

In [ ]:
#Model training 
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 10s 40ms/step - loss: 0.3274 - accuracy: 0.8456 - val_loss: 0.1958 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 7s 38ms/step - loss: 0.1472 - accuracy: 0.9415 - val_loss: 0.1978 - val_accuracy: 0.9167
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.1084 - accuracy: 0.9580 - val_loss: 0.2346 - val_accuracy: 0.9137
Epoch 4/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0794 - accuracy: 0.9709 - val_loss: 0.2622 - val_accuracy: 0.9060
Epoch 5/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0617 - accuracy: 0.9784 - val_loss: 0.3051 - val_accuracy: 0.9022
Epoch 6/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0425 - accuracy: 0.9868 - val_loss: 0.3519 - val_accuracy: 0.9092
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0310 - accuracy: 0.9904 - val_loss: 0.3808 - val_accuracy: 0.900

In [ ]:
y_pred = model.predict(X_test)
y_test, y_test.shape

(array([1, 0, 0, ..., 0, 1, 1]), (6035,))

In [ ]:
y_pred.shape

(6035, 1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred = [int(x) for x in y_pred]
y_pred = np.array(y_pred)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5668599834299917